## 前處理統一版本 (from 彥文)

# LSTM using glove by Sky
## Grid search each settings version 3
## Unified：
1. 字母小寫
2. 刪除網址
3. 移除標點符號(所有標點符號都要刪除)
4. 移除非英文字母
5. 要做class_weight
6. 要做stop_words
7. replace_username：False
8. replace_covid：False
## To be ensured：
1. batch size:64、128、256
2. pooling method：max、average、hidden
3. activation function：sigmoid、ReLU、Softmax
### 2025/5/27 19:00 pm

In [1]:
# 載入套件
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer, word_tokenize
import re
import string
import contractions
from collections import Counter
from itertools import product
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns

# 下載 NLTK 資源
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\skych\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\skych\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\skych\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\skych\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\skych\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\skych\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-d

True

In [3]:
# 檢查 GPU 可用性並設置設備
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用設備：{device}")
print(f"PyTorch 版本：{torch.__version__}")
print(f"GPU 可用：{torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 名稱：{torch.cuda.get_device_name(0)}")

使用設備：cuda
PyTorch 版本：2.5.1+cu121
GPU 可用：True
GPU 名稱：NVIDIA GeForce RTX 4060 Laptop GPU


In [4]:
# 初始化工具
tokenizer = TweetTokenizer(preserve_case=False)
encoder = LabelEncoder()
custom_stopwords = set(stopwords.words('english')) - {"not", "no", "never"}

# 定義資料清理函數
def clean_text(text, use_stopwords=False, replace_username=True, replace_covid='none'):
    if pd.isna(text):
        return []
    
    # 小寫
    text = text.lower()
    
    # 展開縮寫
    text = contractions.fix(text)
    
    # 移除網址
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    
    # 處理@人名
    if replace_username:
        text = re.sub(r"@\w+", 'username', text)
    else:
        text = re.sub(r"@\w+", '', text)
    
    # 處理 covid 相關詞
    if replace_covid == 'virus':
        text = re.sub(r"\bcovid\b|\bcovid19\b|\bcoronavirus\b", 'virus', text, flags=re.IGNORECASE)
    elif replace_covid == 'pandemic':
        text = re.sub(r"\bcovid\b|\bcovid19\b|\bcoronavirus\b", 'pandemic', text, flags=re.IGNORECASE)
    
    # 保留字母
    text = re.sub(r"[^a-zA-Z \s]", '', text)
    
    # 去除多餘空白
    text = re.sub(r"\s+", " ", text).strip()
    
    # 分詞
    tokens = word_tokenize(text)
    
    # 移除停用詞
    if use_stopwords:
        tokens = [word for word in tokens if word not in custom_stopwords]
    
    return tokens

In [5]:
# 讀取資料
df_train = pd.read_csv('Corona_NLP_train.csv', encoding='latin_1')
df_test = pd.read_csv('Corona_NLP_test.csv', encoding='latin_1')
df_train, df_val = train_test_split(df_train, test_size=0.2, stratify=df_train["Sentiment"], random_state=42)

# 顯示資料統計
print("訓練集資料筆數：", len(df_train))
print("訓練集欄位：", df_train.columns.tolist())
print("訓練集情緒分布：\n", df_train["Sentiment"].value_counts())
print("驗證集資料筆數：", len(df_val))
print("驗證集欄位：", df_val.columns.tolist())
print("驗證集情緒分布：\n", df_val["Sentiment"].value_counts())
print("測試集資料筆數：", len(df_test))
print("測試集欄位：", df_test.columns.tolist())
print("測試集情緒分布：\n", df_test["Sentiment"].value_counts())

# 套用前處理函數（初始清理，無停用詞）
df_train["clean_tokens"] = df_train["OriginalTweet"].apply(lambda x: clean_text(x, use_stopwords=False))
df_val["clean_tokens"] = df_val["OriginalTweet"].apply(lambda x: clean_text(x, use_stopwords=False))
df_test["clean_tokens"] = df_test["OriginalTweet"].apply(lambda x: clean_text(x, use_stopwords=False))

# 檢查空序列並移除
empty_train = df_train[df_train['clean_tokens'].apply(len) == 0]
if not empty_train.empty:
    print(f"警告：訓練集有 {len(empty_train)} 筆空序列，移除中...")
    print("空序列範例：")
    print(empty_train[['OriginalTweet', 'clean_tokens']].head())
    df_train = df_train[df_train['clean_tokens'].apply(len) > 0]

empty_val = df_val[df_val['clean_tokens'].apply(len) == 0]
if not empty_val.empty:
    print(f"警告：驗證集有 {len(empty_val)} 筆空序列，移除中...")
    print("空序列範例：")
    print(empty_val[['OriginalTweet', 'clean_tokens']].head())
    df_val = df_val[df_val['clean_tokens'].apply(len) > 0]

empty_test = df_test[df_test['clean_tokens'].apply(len) == 0]
if not empty_test.empty:
    print(f"警告：測試集有 {len(empty_test)} 筆空序列，移除中...")
    print("空序列範例：")
    print(empty_test[['OriginalTweet', 'clean_tokens']].head())
    df_test = df_test[df_test['clean_tokens'].apply(len) > 0]

# 處理情緒標籤
df_train["SentimentEncoded"] = encoder.fit_transform(df_train["Sentiment"])
df_val["SentimentEncoded"] = encoder.transform(df_val["Sentiment"])
df_test["SentimentEncoded"] = encoder.transform(df_test["Sentiment"])

# 顯示處理結果
print("\n訓練集處理結果：")
print(df_train[["OriginalTweet", "clean_tokens", "Sentiment", "SentimentEncoded"]].head())
print("\n驗證集處理結果：")
print(df_val[["OriginalTweet", "clean_tokens", "Sentiment", "SentimentEncoded"]].head())
print("\n測試集處理結果：")
print(df_test[["OriginalTweet", "clean_tokens", "Sentiment", "SentimentEncoded"]].head())

訓練集資料筆數： 32925
訓練集欄位： ['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet', 'Sentiment']
訓練集情緒分布：
 Positive              9137
Negative              7934
Neutral               6170
Extremely Positive    5299
Extremely Negative    4385
Name: Sentiment, dtype: int64
驗證集資料筆數： 8232
驗證集欄位： ['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet', 'Sentiment']
驗證集情緒分布：
 Positive              2285
Negative              1983
Neutral               1543
Extremely Positive    1325
Extremely Negative    1096
Name: Sentiment, dtype: int64
測試集資料筆數： 3798
測試集欄位： ['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet', 'Sentiment']
測試集情緒分布：
 Negative              1041
Positive               947
Neutral                619
Extremely Positive     599
Extremely Negative     592
Name: Sentiment, dtype: int64
警告：訓練集有 10 筆空序列，移除中...
空序列範例：
                                           OriginalTweet clean_tokens
29888  ???? ????? \r\r\n????? ??? ? ?? ?? ??\r\r\n\r\...           []
2

In [6]:
# 定義詞彙表構建函數
def build_vocab(token_lists):
    all_tokens = [token for tokens in token_lists for token in tokens]
    token_counts = Counter(all_tokens)
    vocab = {'<PAD>': 0, '<UNK>': 1}
    vocab.update({token: idx + 2 for idx, (token, _) in enumerate(token_counts.items())})
    return vocab

# 定義資料集類
class TweetDataset(Dataset):
    def __init__(self, df, vocab, max_length):
        self.texts = df['clean_tokens']
        self.labels = df['SentimentEncoded'].values
        self.vocab = vocab
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokens = self.texts.iloc[idx]
        indices = [self.vocab.get(token, self.vocab['<UNK>']) for token in tokens]
        if len(indices) > self.max_length:
            indices = indices[:self.max_length]
        else:
            indices += [self.vocab['<PAD>']] * (self.max_length - len(indices))
        return torch.tensor(indices, dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)

# 定義載入 Glove 嵌入函數
def load_glove_embeddings(glove_path, vocab, embedding_dim=100):
    embeddings = np.zeros((len(vocab), embedding_dim))
    with open(glove_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            if word in vocab:
                embeddings[vocab[word]] = np.array(values[1:], dtype=np.float32)
    embeddings[vocab['<PAD>']] = np.zeros(embedding_dim)
    embeddings[vocab['<UNK>']] = np.random.uniform(-0.25, 0.25, embedding_dim)
    return embeddings

In [8]:
# 定義 BiLSTM 模型
class BiLSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout, pooling_method, activation):
        super(BiLSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, bidirectional=True, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.pooling_method = pooling_method
        self.activation = activation
        if activation == 'sigmoid':
            self.act_fn = nn.Sigmoid()
        elif activation == 'relu':
            self.act_fn = nn.ReLU()
        elif activation == 'softmax':
            self.act_fn = nn.Softmax(dim=1)
        else:
            raise ValueError("Invalid activation function. Choose 'sigmoid', 'relu', or 'softmax'.")

    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        lstm_out, (hidden, cell) = self.lstm(embedded)
        if self.pooling_method == 'hidden':
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        elif self.pooling_method == 'max':
            hidden, _ = torch.max(lstm_out, dim=1)
        elif self.pooling_method == 'average':
            hidden = torch.mean(lstm_out, dim=1)
        else:
            raise ValueError("Invalid pooling method. Choose 'hidden', 'max', or 'average'.")
        output = self.fc(hidden)
        if self.activation in ['sigmoid', 'relu']:
            output = self.act_fn(output)
        return output  # Softmax 由損失函數處理或在 forward 中應用

import logging  # 添加 logging 導入
# 設定日誌
logging.basicConfig(
    filename='training_log_0527.txt',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [9]:
# 網格搜索
batch_sizes = [64, 128, 256]
pooling_methods = ['hidden', 'max', 'average']
activation_functions = ['sigmoid', 'relu', 'softmax']
results = []
best_val_acc = 0
best_params = None
best_train_losses = None
best_val_losses = None
best_val_accuracies = None

total_combinations = len(batch_sizes) * len(pooling_methods) * len(activation_functions)
current_combination = 0

progress_file = 'progress_0527.txt'

In [13]:
from datetime import datetime
def update_progress(current, total):
    percentage = (current / total) * 100
    with open(progress_file, 'w') as f:
        f.write(f"Current Combination: {current}/{total}\n")
        f.write(f"Progress: {percentage:.2f}%\n")
        f.write(f"Last Updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

In [14]:
for batch_size, pooling_method, activation in product(batch_sizes, pooling_methods, activation_functions):
    current_combination += 1
    start_time = datetime.now()
    log_msg = (f"試驗 {current_combination}/{total_combinations}：batch_size={batch_size}, "
               f"pooling_method={pooling_method}, activation={activation}")
    print(log_msg)
    logging.info(log_msg)
    update_progress(current_combination, total_combinations)

    # 構建詞彙表和資料集
    vocab = build_vocab(df_train['clean_tokens'])
    train_dataset = TweetDataset(df_train, vocab, max_length=50)  # 固定 max_length=50
    val_dataset = TweetDataset(df_val, vocab, max_length=50)
    test_dataset = TweetDataset(df_test, vocab, max_length=50)

    # 建立 DataLoader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, pin_memory=True)

    # 初始化模型
    model = BiLSTMClassifier(
        vocab_size=len(vocab),
        embedding_dim=100,
        hidden_dim=256,
        output_dim=5,
        n_layers=4,
        dropout=0.2,
        pooling_method=pooling_method,
        activation=activation
    ).to(device)

    # 載入 Glove 嵌入
    glove_embeddings = load_glove_embeddings(r'C:\Users\skych\glove.twitter.27B\glove.twitter.27B.100d.txt', vocab)
    model.embedding.weight.data.copy_(torch.tensor(glove_embeddings, device=device))
    model.embedding.weight.requires_grad = False

    # 設定優化器和損失函數
    optimizer = optim.Adam(model.parameters())
    class_weights = torch.tensor(
        compute_class_weight('balanced', classes=np.unique(df_train['Sentiment']), y=df_train['Sentiment']),
        device=device
    ).float()
    criterion = nn.CrossEntropyLoss(weight=class_weights).to(device)

    # 訓練模型
    num_epochs = 10
    patience = 3
    current_val_acc = 0
    counter = 0
    train_losses, val_losses, val_accuracies = [], [], []

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            texts, labels = batch
            texts, labels = texts.to(device), labels.to(device)
            optimizer.zero_grad()
            predictions = model(texts)
            if model.activation == 'softmax':
                loss = criterion(predictions, labels)  # Softmax 已在 forward 中處理
            else:
                loss = criterion(predictions, labels)  # CrossEntropyLoss 內含 log_softmax
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        train_losses.append(epoch_loss / len(train_loader))

        # 驗證
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch in val_loader:
                texts, labels = batch
                texts, labels = texts.to(device), labels.to(device)
                predictions = model(texts)
                if model.activation == 'softmax':
                    loss = criterion(predictions, labels)
                else:
                    loss = criterion(predictions, labels)
                val_loss += loss.item()
                _, predicted = torch.max(predictions, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_losses.append(val_loss / len(val_loader))
        val_accuracies.append(correct / total)

        # 記錄 epoch 結果
        log_msg = (f"Epoch {epoch+1}/{num_epochs} 完成：訓練損失={train_losses[-1]:.4f}, "
                   f"驗證損失={val_losses[-1]:.4f}, 驗證準確率={val_accuracies[-1]:.4f}")
        print(log_msg)
        logging.info(log_msg)

        # 提前停止
        if val_accuracies[-1] > current_val_acc:
            current_val_acc = val_accuracies[-1]
            counter = 0
        else:
            counter += 1
        if counter >= patience:
            log_msg = f"提前停止於 epoch {epoch+1}"
            print(log_msg)
            logging.info(log_msg)
            break

    # 記錄結果
    results.append({
        'batch_size': batch_size,
        'pooling_method': pooling_method,
        'activation': activation,
        'best_val_accuracy': current_val_acc
    })

    # 更新最佳參數
    if current_val_acc > best_val_acc:
        best_val_acc = current_val_acc
        best_params = {
            'batch_size': batch_size,
            'pooling_method': pooling_method,
            'activation': activation
        }
        best_train_losses = train_losses
        best_val_losses = val_losses
        best_val_accuracies = val_accuracies

    # 記錄組合完成
    log_msg = f"試驗 {current_combination}/{total_combinations} 完成，耗時：{datetime.now() - start_time}"
    print(log_msg)
    logging.info(log_msg)

    # 釋放 GPU 記憶體
    torch.cuda.empty_cache()

試驗 2/27：batch_size=64, pooling_method=hidden, activation=sigmoid


Epoch 1/10: 100%|██████████| 515/515 [00:16<00:00, 31.91it/s]


Epoch 1/10 完成：訓練損失=1.5683, 驗證損失=1.5739, 驗證準確率=0.2399


Epoch 2/10: 100%|██████████| 515/515 [00:14<00:00, 34.72it/s]


Epoch 2/10 完成：訓練損失=1.5782, 驗證損失=1.5512, 驗證準確率=0.2538


...

Epoch 8/10 完成：訓練損失=0.7540, 驗證損失=0.7239, 驗證準確率=0.7111


Epoch 9/10: 100%|██████████| 129/129 [00:08<00:00, 14.61it/s]


Epoch 9/10 完成：訓練損失=0.7150, 驗證損失=0.6963, 驗證準確率=0.7235


Epoch 10/10: 100%|██████████| 129/129 [00:08<00:00, 14.69it/s]


Epoch 10/10 完成：訓練損失=0.6687, 驗證損失=0.6693, 驗證準確率=0.7421
試驗 28/27 完成，耗時：0:01:41.629032


In [1]:
# 顯示並分析結果
results_df = pd.DataFrame(results)
log_msg = "\nGrid Search 結果：\n" + str(results_df)
print(log_msg)
logging.info(log_msg)
results_df.to_csv('grid_search_results_0527.csv', index=False)

# 報告最佳參數
log_msg = (f"\n最佳參數組合：\n"
           f"batch_size: {best_params['batch_size']}\n"
           f"pooling_method: {best_params['pooling_method']}\n"
           f"activation: {best_params['activation']}\n"
           f"最佳驗證準確率: {best_val_acc:.4f}")
print(log_msg)
logging.info(log_msg)

NameError: name 'pd' is not defined

In [ ]:
# 儲存最佳模型的訓練曲線
plt.rcParams['font.sans-serif'] = ['SimHei']  # 支援中文
plt.rcParams['axes.unicode_minus'] = False
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(best_train_losses)+1), best_train_losses, label='訓練損失')
plt.plot(range(1, len(best_val_losses)+1), best_val_losses, label='驗證損失')
plt.xlabel('Epoch')
plt.ylabel('損失')
plt.title('最佳模型：訓練/驗證損失')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(range(1, len(best_val_accuracies)+1), best_val_accuracies, label='驗證準確率', color='green')
plt.xlabel('Epoch')
plt.ylabel('準確率')
plt.title('最佳模型：驗證準確率')
plt.legend()
plt.tight_layout()
plt.savefig('best_model_training_curves_0527.png')
plt.close()
logging.info("最佳模型訓練曲線已儲存至 'best_model_training_curves_0527.png'")